### Import

In [1]:
import pandas as pd
import numpy as np
import pickle

### Load the json data

In [2]:
list_df = []
for i in range (10):
    list_df.append(pd.read_json(f'../cluster_data/0{i}.json', lines=True))
for i in (np.arange(10, 60, 1)):
    list_df.append(pd.read_json(f'../cluster_data/{i}.json', lines=True))

df = pd.concat(list_df)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  import sys


### Create a dictionary containing each tweet with the tag 

In [17]:
list_hashtags = []
dict_tot = dict()
for i in range(df.text.index.max()):
    dict_tmp = dict()
    if (type(df.text[i])==str):
        if (len(df.entities[i]['hashtags'])!=0):
            dict_tmp['hashtags'] = df.entities[i]['hashtags'][0]['text']
            dict_tmp['text'] = df.text[i]
        else :
            dict_tmp['hashtags'] = df.entities[i]['hashtags']
            dict_tmp['text'] = df.text[i]
        dict_tot[(i,-1)]=dict_tmp
    elif type(df.text[i])==pd.core.series.Series: 
        for j in range(len(df.text[i])): 
            dict_tmp = dict()
            if (type(df.text[i].reset_index().text[j])==str):
                if len(df.entities[i].reset_index().entities[j]['hashtags'])!=0:
                    dict_tmp['hashtags'] = df.entities[i].reset_index().entities[j]['hashtags'][0]['text']
                    dict_tmp['text'] = df.text[i].reset_index()['text'][j]
                else : 
                    dict_tmp['hashtags'] = df.entities[i].reset_index().entities[j]['hashtags']
                    dict_tmp['text'] = df.text[i].reset_index()['text'][j]
                dict_tot[(i,j)]=dict_tmp
pickle.dump(dict_tot, open('dictionary_with_all_tweets_and_hashtags.p', 'wb'))

In [18]:
dict_tot = pickle.load(open('dictionary_with_all_tweets_and_hashtags.p', 'rb'))

### Load the list of investor and compagny we are working on 

In [19]:
compagny = pickle.load(open('compagny.p', 'rb'))
investor = pickle.load(open('investor.p', 'rb'))
############# ATTENTION IF FAUT QUE JE CLEAN CES NOMs

In [33]:
# Hard code data and try on a small subset
compagny = compagny[0:5]
investor = investor[0:5]
dict_tot[(0,0)]['text'] = 'ALTABA INC is a really nice compagny'
dict_tot[(0,1)]['text'] = 'I am never disappointed with ALTABA INC'
dict_tot[(0,2)]['text'] = 'ALTABA INC have nice product'
dict_tot[(0,3)]['text'] = 'ALTABA INC <3 :)'
dict_tot[(0,4)]['text'] = 'good bro !! ALTABA INC '
dict_tot[(0,5)]['text'] = 'CATERPILLAR INC DEL will kill the world'
dict_tot[(0,6)]['text'] = 'CATERPILLAR INC DEL is really bad'
dict_tot[(0,7)]['text'] = 'I am really angry against CATERPILLAR INC DEL '
dict_tot[(0,8)]['text'] = 'Shit !!! CATERPILLAR INC DEL '
dict_tot[(0,9)]['text'] = 'CATERPILLAR INC DEL :( :( :('
dict_tot[(0,10)]['text'] = 'SPDR S&P 500 ETF TR sell some vegetables'
dict_tot[(0,11)]['text'] = 'SPDR S&P 500 ETF TR play football'
dict_tot[(0,12)]['text'] = 'SPDR S&P 500 ETF TR is located in lausanne'
dict_tot[(0,13)]['text'] = 'SPDR S&P 500 ETF TR is in another country'
dict_tot[(0,14)]['text'] = '__ __ SPDR S&P 500 ETF TR'

KeyError: (0, 0)

### Create a dictionary using the compagny or investor as key grouping all the tweets about it
We consider that the tweet concerned a compagny/investor is there is his name in the tweet or the hashtag

In [22]:
def tweet_per_category(list_of_names, dict_tot):
    dict_per_category = dict()

    for name in list_of_names :
        list_tweet = []
        for key in list(dict_tot.keys()) :
            if (str(dict_tot[key]['hashtags']).lower().find(name.lower()) != -1) | (dict_tot[key]['text'].lower().find(name.lower()) != -1):
                list_tweet.append(dict_tot[key]['text'])
        dict_per_category[name] = list_tweet
    return dict_per_category

In [23]:
dict_per_compagny = tweet_per_category(compagny, dict_tot)
pickle.dump(dict_per_compagny, open('dictionary_per_compagny_tweet.p', 'wb'))

In [24]:
dict_per_compagny

{'ALTABA INC': ['ALTABA INC is a really nice compagny',
  'I am never disappointed with ALTABA INC',
  'ALTABA INC have nice product',
  'ALTABA INC <3 :)',
  'good bro !! ALTABA INC '],
 'SPDR S&P 500 ETF TR': [],
 'CATERPILLAR INC DEL': ['CATERPILLAR INC DEL will kill the world',
  'CATERPILLAR INC DEL is really bad',
  'I am really angry against CATERPILLAR INC DEL ',
  'Shit !!! CATERPILLAR INC DEL ',
  'CATERPILLAR INC DEL :( :( :('],
 'DOWDUPONT INC': [],
 'INVESCO QQQ TR': []}

In [25]:
dict_per_investor = tweet_per_category(investor, dict_tot)
pickle.dump(dict_per_investor, open('dictionary_per_investor_tweet.p', 'wb'))

In [26]:
dict_per_investor

{'KINGDON CAPITAL MANAGEMENT, L.L.C.': [],
 'ROYAL BANK OF CANADA': [],
 'GIRARD PARTNERS LTD.': [],
 'SANDLER CAPITAL MANAGEMENT': [],
 'BROOKFIELD ASSET MANAGEMENT INC.': []}

In [27]:
dict_tot = pickle.load(open('dictionary_per_compagny_tweet.p', 'rb'))

In [28]:
dict_tot = pickle.load(open('dictionary_per_investor_tweet.p', 'rb'))

### Make sentiment analysis for each tweet for each compagny/investors :

#### Vader
This library takes into account :
- positive sentence example
- punctuation emphasis handled correctly (sentiment intensity adjusted)
- booster words handled correctly (sentiment intensity adjusted)
- emphasis for ALLCAPS handled
- combination of signals - VADER appropriately adjusts intensity
-  booster words & punctuation make this close to ceiling for score
- negation sentence example
- positive sentence
- negated negative sentence with contraction
- qualified positive sentence is handled correctly (intensity adjusted)
- mixed negation sentence
- negative slang with capitalization emphasis
- mixed sentiment example with slang and constrastive conjunction "but"
- emoticons handled
- emojis handled
- Capitalized negation

In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [30]:
def vader_analysis(dict_per_):
    analyzer = SentimentIntensityAnalyzer()
    dict_score = dict()
    for key in list(dict_per_.keys()):
        neg, pos, neu, compound, tmp_dict = [], [], [], [], dict()
        for sentence in dict_per_[key]:
            vs = analyzer.polarity_scores(sentence)
            neg.append(vs['neg'])
            neu.append(vs['neu'])
            pos.append(vs['pos'])
            compound.append(vs['compound'])
        tmp_dict['neg'] = np.mean(neg)
        tmp_dict['pos'] = np.mean(pos)
        tmp_dict['neu'] = np.mean(neu)
        tmp_dict['compound'] = np.mean(compound)
        dict_score[key] = tmp_dict 
        
    return dict_score

In [31]:
output = vader_analysis(dict_per_compagny)
pickle.dump(output, open('dictionary_per_compagny_score_vader.p', 'wb'))
output

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'ALTABA INC': {'neg': 0.0,
  'pos': 0.46900000000000003,
  'neu': 0.531,
  'compound': 0.50376},
 'SPDR S&P 500 ETF TR': {'neg': nan, 'pos': nan, 'neu': nan, 'compound': nan},
 'CATERPILLAR INC DEL': {'neg': 0.5034000000000001,
  'pos': 0.0,
  'neu': 0.49659999999999993,
  'compound': -0.6654},
 'DOWDUPONT INC': {'neg': nan, 'pos': nan, 'neu': nan, 'compound': nan},
 'INVESCO QQQ TR': {'neg': nan, 'pos': nan, 'neu': nan, 'compound': nan}}

In [35]:
output = vader_analysis(dict_per_investor)
pickle.dump(output, open('dictionary_per_investor_score_vader.p', 'wb'))
output

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'KINGDON CAPITAL MANAGEMENT, L.L.C.': {'neg': nan,
  'pos': nan,
  'neu': nan,
  'compound': nan},
 'ROYAL BANK OF CANADA': {'neg': nan, 'pos': nan, 'neu': nan, 'compound': nan},
 'GIRARD PARTNERS LTD.': {'neg': nan, 'pos': nan, 'neu': nan, 'compound': nan},
 'SANDLER CAPITAL MANAGEMENT': {'neg': nan,
  'pos': nan,
  'neu': nan,
  'compound': nan},
 'BROOKFIELD ASSET MANAGEMENT INC.': {'neg': nan,
  'pos': nan,
  'neu': nan,
  'compound': nan}}

#### FastText

In [36]:
import fastText
import re
import string
import os

In [37]:
MODEL_DIR_PATH = "../../Downloads/"

model = fastText.load_model(os.path.join(MODEL_DIR_PATH, "amazon_review_full.bin"))

maketrans = str.maketrans

In [38]:
def clean_text(text):
    """
    Applies some pre-processing to clean text data.
    
    In particular:
    - lowers the string
    - removes the character [']
    - replaces punctuation characters with spaces

    """
    
    text = text.lower()

    text = re.sub(r"\'", "", text)  # remove the character [']

    # removing the punctuation
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    split = " "

    if isinstance(text, str):
        translate_map = dict((ord(c), str(split)) for c in filters)
        text = text.translate(translate_map)
    elif len(split) == 1:
        translate_map = maketrans(filters, split * len(filters))
        text = text.translate(translate_map)
    else:
        for c in filters:
            text = text.replace(c, split)
    return text

In [39]:
predict_sentiment = lambda s: model.predict(clean_text(s))

In [42]:
def fastext_prediction(dict_per_):
    dict_score = dict()
    for key in list(dict_per_.keys()):
        label, confidence, tmp_dict = [], [], dict()
        for sentence in dict_per_[key]:
            res = predict_sentiment(sentence)
            label.append(int(res[0][0][9]))
            confidence.append(res[1][0])
        tmp_dict['label'] = np.mean(label)
        tmp_dict['confidence'] = np.mean(confidence)
        dict_score[key] = tmp_dict 

    return dict_score

In [44]:
output = fastext_prediction(dict_per_compagny)
pickle.dump(output, open('dictionary_per_compagny_score_fastext.p', 'wb'))
output

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'ALTABA INC': {'label': 3.0, 'confidence': 0.8472848296165466},
 'SPDR S&P 500 ETF TR': {'label': nan, 'confidence': nan},
 'CATERPILLAR INC DEL': {'label': 1.8, 'confidence': 0.7830009698867798},
 'DOWDUPONT INC': {'label': nan, 'confidence': nan},
 'INVESCO QQQ TR': {'label': nan, 'confidence': nan}}

In [46]:
output = fastext_prediction(dict_per_investor)
pickle.dump(output, open('dictionary_per_investor_score_fastext.p', 'wb'))
output

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'KINGDON CAPITAL MANAGEMENT, L.L.C.': {'label': nan, 'confidence': nan},
 'ROYAL BANK OF CANADA': {'label': nan, 'confidence': nan},
 'GIRARD PARTNERS LTD.': {'label': nan, 'confidence': nan},
 'SANDLER CAPITAL MANAGEMENT': {'label': nan, 'confidence': nan},
 'BROOKFIELD ASSET MANAGEMENT INC.': {'label': nan, 'confidence': nan}}